In [1]:
import covid_19_shared as COVID_19

import pandas as pd

def get_all_states_stats(verbose : bool = True):
    if (verbose):
        print("Getting population...")
    pop = COVID_19.USA_population()

    if (verbose): print("Getting list of states...")
    states = COVID_19.list_of_JHUD_US_states()

    if (verbose): print("Getting USA data...")
    us_data = COVID_19.USA_Data()

    if (verbose): print("%-30s%-10s%-20s%-10s%-16s" 
          % ("State", "New Conf.", "Conf./Popul.(%)"
             , "New Death", "Death/Popul.(%)")
         )
    df = pd.DataFrame()
    for state in states:
        if state in pop:

            stats = COVID_19.COVID_19_Stats(state, COVID_19.usa_state_data(state, us_data))
            state_pop = int(pop[state])
            stats.pops_ratio_conf_ = stats.cv_data_.Confirmed * 100 / state_pop
            stats.pops_ratio_deaths_ = stats.cv_data_.Dead * 100 / state_pop
            if (verbose): print("%-30s%-10d%-20f%-10d%-16f" 
                  % (state
                        , int(stats.confirmed_new_[-1])
                        , stats.pops_ratio_conf_[-1]
                        , int(stats.deaths_new_[-1])
                        , stats.pops_ratio_deaths_[-1]
                       )
                 )
            df = df.append(pd.DataFrame({"State" : [state], "Statistics": [stats], "Population" : [state_pop]}))
    #         stats.dynamics()

    return df.set_index("State")

In [2]:
df = get_all_states_stats()

Getting population...
Getting list of states...
Getting USA data...
State                         New Conf. Conf./Popul.(%)     New Death Death/Popul.(%) 
Alabama                       2288      5.646274            12        0.081364        
Alaska                        765       5.214641            0         0.020134        
Arizona                       5376      5.698921            25        0.108729        
Arkansas                      1542      5.861756            40        0.091223        
California                    28732     3.668531            58        0.053492        
Colorado                      3234      5.181365            -1        0.066730        
Connecticut                   0         3.573350            0         0.143980        
Delaware                      816       4.444870            11        0.088314        
District of Columbia          264       3.844959            2         0.115834        
Florida                       8436      5.627661            93

In [3]:

df["PopulShare"] =df.Population * 100 / df.Population.sum()

df["Confirmed"] = pd.Series(index=df.index, data=[(v.cv_data_.Confirmed[-1]) for v in df["Statistics"]])
df["OneDayConf"] = pd.Series(index=df.index
                             , data=[v.cv_data_.Confirmed[-1] - v.cv_data_.Confirmed[-2] 
                                     for v in df["Statistics"]])
df["OneWeekConfirmed"] = pd.Series(index=df.index, data=[(v.cv_data_.Confirmed[-1] - v.cv_data_.Confirmed[-8]) for v in df["Statistics"]])

df["ConfShare"] = df.Confirmed * 100 / df.Confirmed.sum()
df["NewConfShare"] = df.OneWeekConfirmed * 100 / df.OneWeekConfirmed.sum()  # using one week stats

df["Deaths"] = pd.Series(index=df.index, data=[(v.cv_data_.Dead[-1]) for v in df["Statistics"]])
df["OneDayDeaths"] = pd.Series(index=df.index, data=[v.cv_data_.Dead[-1] - v.cv_data_.Dead[-2] for v in df["Statistics"]])
df["OneWeekDeaths"] = pd.Series(index=df.index, data=[v.cv_data_.Dead[-1] - v.cv_data_.Dead[-8] for v in df["Statistics"]])

df["DeathsShare"] = df.Deaths * 100 / df.Deaths.sum()
df["NewDeathsShare"] = df.OneWeekDeaths * 100 / df.OneWeekDeaths.sum()

# Indicator of intensity
# "Density" as ratio of value share to population share
df["CasesDensity"] = df.ConfShare / df.PopulShare
df["NewCasesDensity"] = df.NewConfShare / df.PopulShare

df["DeathsDensity"] = df.DeathsShare / df.PopulShare
df["NewDeathsDensity"] = df.NewDeathsShare / df.PopulShare


In [4]:
# df3 = df.copy()
# del df3["Statistics"]
df3 = df[["OneDayConf"
          ,"OneWeekConfirmed"
          , "OneWeekDeaths"
          
          , "CasesDensity"
          , "NewCasesDensity"
          
          , "DeathsDensity"
          , "NewDeathsDensity"

          , "NewDeathsShare"
         ]]
# dropping statistics column 
df3 = df3.sort_values('NewDeathsShare', ascending=False)
df3

,OneDayConf,OneWeekConfirmed,OneWeekDeaths,CasesDensity,NewCasesDensity,DeathsDensity,NewDeathsDensity,NewDeathsShare
State,,,,,,,,
Texas,11095,97620,1294,1.114513,0.883267,1.018975,1.042457,8.388979
Illinois,7598,67459,1234,1.300515,1.196207,1.218376,1.948283,8.000000
Pennsylvania,7465,62156,958,0.705717,1.113300,0.981247,1.527796,6.210697
Michigan,0,48424,854,0.914434,1.114700,1.156439,1.750354,5.536467
California,28732,151218,777,0.777258,0.923518,0.592392,0.422507,5.037277
Florida,8436,65414,677,1.192341,0.791584,1.129563,0.729433,4.388979
Ohio,7592,60592,560,0.872397,1.194966,0.668021,0.983329,3.630470
Indiana,6598,48305,557,1.247012,1.695027,1.066134,1.740245,3.611021
Wisconsin,3149,32013,465,1.643218,1.280733,0.769578,1.656364,3.014587
